In [1]:
# install dependencies
! pip install langchain chromadb huggingface_hub sentence-transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import userdata
HUGGINGFACEHUB_API_TOKEN = userdata.get('HUGGINGFACE_TOKEN')

In [28]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [4]:
# download & unzip dataset
!wget -q https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip
!unzip -q new_articles.zip -d new_articles

In [5]:
# importing chroma and huggingface embedding from langchain
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader

In [6]:
loader = DirectoryLoader('./new_articles/', glob="./*.txt", loader_cls=TextLoader)
documents = loader.load()

In [7]:
len(documents)

21

In [8]:
# chunking operation
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [9]:
# creating vector DB
persist_directory = 'db'
embedding = HuggingFaceEmbeddings()
vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
# persiste the db to disk
vectordb.persist()
vectordb = None

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [11]:
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

In [12]:
# semantic search
retriever = vectordb.as_retriever()
docs = retriever.get_relevant_documents("what is spacex?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [13]:
docs

[Document(page_content='SpaceX’s super-heavy launch system Starship is poised to fundamentally reshape the space economy. The 394-foot-tall vehicle, which took to the skies for the first time last month, is designed to carry a staggering amount of mass to low Earth orbit and into deep space.\n\nTechCrunch+ spoke with three pure-play space VCs — Space Capital founder and managing partner Chad Anderson, Space.VC founder and general partner Jonathan Lacoste and E2MC Ventures founder Raphael Roettgen — to learn more about how they advise founders to think through Starship’s super-heavy implications.\n\nWhile the trio diverges on many fine points, they all agreed that founders should be thinking now about how Starship could affect their operations, for better or worse.\n\n“Starship has such high importance to the space sector that probably almost everyone who has a space company has to war game what that means for their business,” Roettgen said.\n\nChanging the face of launch …', metadata={

In [14]:
len(docs)

4

In [15]:
# make a chain
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFaceEndpoint

In [38]:
repo_id = "mistralai/Mistral-7B-Instruct-v0.2"

llm = HuggingFaceEndpoint(
    repo_id=repo_id, max_length=20, temperature=0.5, token=HUGGINGFACEHUB_API_TOKEN
)

                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.
                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [39]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [40]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [41]:
# full example
query = "what is spacex?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 SpaceX is a private American aerospace manufacturer and space transportation company founded in 2002 by Elon Musk with the goal of reducing space transportation costs and enabling the colonization of Mars. SpaceX operates from many pads, including two in Florida and one in California. It has developed several launch vehicles and the Dragon spacecraft, which is flown into orbit by the Falcon launch vehicles and is used to deliver cargo and eventually humans to the International Space Station and other destinations. SpaceX's Starship is a newer, even more ambitious project designed to carry a much larger payload to orbit and into deep space.


Sources:
new_articles/05-07-spacex-starship-startups-future.txt
new_articles/05-07-fintech-space-continues-to-be-competitive-and-drama-filled.txt
new_articles/05-05-vint-cerf-on-the-exhilarating-mix-of-thrill-and-hazard-at-the-frontiers-of-tech.txt
new_articles/05-07-spacex-starship-startups-future.txt


In [42]:
# full example
query = "what is chatgpt?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 ChatGPT is a general-purpose chatbot developed by OpenAI that uses artificial intelligence to generate text after a user enters a prompt. It uses a large language model called GPT-4 to produce human-like text. ChatGPT can engage with a range of topics, including programming, TV scripts, scientific concepts, and more. It's free to use, but there's also a paid version called ChatGPT Plus. ChatGPT can be used for everyday tasks like programming, scripts, email replies, listicles, blog ideas, summarization, and more. It can also be used for advanced tasks like debugging code, programming languages, scientific concepts, and complex problem solving. However, it's important to note that ChatGPT lacks context awareness and can't always generate appropriate code or text for a specific context. There have been controversies surrounding ChatGPT's use in SEO farming, plagiarism, and misinformation, as well as accusations of generating false crimes. ChatGPT prompts can be found on various marketpl

In [43]:
# full example
query = "who is databrics?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Databricks is a data analytics platform that provides unified analytics at scale for data science, engineering, and business analytics teams. With Databricks, organizations can process and analyze all their data using SQL, Streaming, MLlib, Spark, and PySpark. The acquisition of Okera will bring advanced data governance capabilities to the Databricks platform.


Sources:
new_articles/05-03-databricks-acquires-ai-centric-data-governance-platform-okera.txt
new_articles/05-03-databricks-acquires-ai-centric-data-governance-platform-okera.txt
new_articles/05-03-databricks-acquires-ai-centric-data-governance-platform-okera.txt
new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt


In [44]:
# full example
query = "who is vignesh?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Vignesh is not mentioned in the context provided.


Sources:
new_articles/05-06-this-week-in-apps-apple-and-google-team-up-on-trackers-google-i-o-preview-apps-hit-newfronts.txt
new_articles/05-06-amazon-launches-free-channels-check-marks-come-to-gmail-and-openai-raises-more-moolah.txt
new_articles/05-04-slack-updates-aim-to-put-ai-at-the-center-of-the-user-experience.txt
new_articles/05-05-vint-cerf-on-the-exhilarating-mix-of-thrill-and-hazard-at-the-frontiers-of-tech.txt


In [45]:
# full example
query = "what launched by amazon?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Amazon launched Amazon Inspire, a new in-app TikTok-like shopping feed for all customers in the United States.


Sources:
new_articles/05-06-this-week-in-apps-apple-and-google-team-up-on-trackers-google-i-o-preview-apps-hit-newfronts.txt
new_articles/05-06-this-week-in-apps-apple-and-google-team-up-on-trackers-google-i-o-preview-apps-hit-newfronts.txt
new_articles/05-06-this-week-in-apps-apple-and-google-team-up-on-trackers-google-i-o-preview-apps-hit-newfronts.txt
new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt
